In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import pdist, squareform
from scipy.stats import norm
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
)
from scikitplot.metrics import plot_roc
from scikitplot.metrics import plot_precision_recall

In [ ]:
df_train = pd.read_csv('train.csv', skipinitialspace=True)

df_test = pd.read_csv('test.csv', skipinitialspace=True)

## Data transformation

In [ ]:
genres = df_train['genre'].unique()
# creo un dizionario per tenerli associati ad un numero
genres_dict = dict(zip(genres, range(len(genres))))
# Mappo i generi con i numeri
df_train['genre'] = df_train['genre'].map(genres_dict)
df_test['genre'] = df_test['genre'].map(genres_dict)
print(genres_dict)

In [ ]:
df_train['explicit'] = df_train['explicit'].astype(int)
df_test['explicit'] = df_test['explicit'].astype(int)

In [1]:
#Calculating only for the missing values in time_signature column by dividing n_beats/n_bars and rounding the result
df_train.loc[df_train['time_signature'].isna(), 'time_signature'] = round(df_train['n_beats'] / df_train['n_bars'])

NameError: name 'df_train' is not defined

In [ ]:
scaler = MinMaxScaler()

# Normalizing duration_ms
df_train['duration_ms'] = scaler.fit_transform(df_train['duration_ms'].values.reshape(-1, 1))
# Normalizing features_duration_ms
df_train['features_duration_ms'] = scaler.fit_transform(df_train['features_duration_ms'].values.reshape(-1, 1))
# Normalizing n_bars
df_train['n_bars'] = scaler.fit_transform(df_train['n_bars'].values.reshape(-1, 1))
# Normalizing n_beats
df_train['n_beats'] = scaler.fit_transform(df_train['n_beats'].values.reshape(-1, 1))
# Normalizing loudness
df_train['loudness'] = scaler.fit_transform(df_train['loudness'].values.reshape(-1, 1))
# Normalizing tempo
df_train['tempo'] = scaler.fit_transform(df_train['tempo'].values.reshape(-1, 1))
# Normalizing processing
df_train['processing'] = scaler.fit_transform(df_train['processing'].values.reshape(-1, 1))

In [ ]:
#Dropping popularity confidence column
df_train.drop(columns=['popularity_confidence'], axis=1, inplace=True)

In [ ]:
# Load your dataset
df_copy = df_train.copy()

df_copy.drop(columns=['name', 'artists', 'album_name'], axis=1, inplace=True)

#divide the dataset in 2 parts: one with known mode and one with missing mode
data_with_mode = df_copy.dropna(subset=['mode'])
data_missing_mode = df_copy[df_copy['mode'].isna()]

# Features and target variable for the known 'mode'
X_known = data_with_mode.drop(columns=['mode'])
y_known = data_with_mode['mode']

# Features for the rows with missing 'mode'
X_missing = data_missing_mode.drop(columns=['mode'])

# Train a classifier (e.g., Random Forest) on data with known 'mode'
clf = RandomForestClassifier(n_estimators=500, random_state=42, max_features=20)
clf.fit(X_known, y_known)

# Use the trained model to predict 'mode' for rows with missing 'mode'
y_missing_pred = clf.predict(X_missing)

# Fill in the missing 'mode' values with predictions
df_train.loc[df_train['mode'].isna(), 'mode'] = y_missing_pred

## Data classification

### Partitioning 

In [ ]:
from sklearn.model_selection import train_test_split
